# Module: Fetch Data

In [1]:
from typing import Union
import pandas as pd
import requests
from datetime import datetime
import pytse_client as tse
import fetch
import config
from utils import debug_log, data_log, ar_to_fa_series, ar_to_fa, fa_to_ar, fa_to_ar_series, flatten_json

pd.set_option('display.max_columns', 30) # pd.options.display.max_columns = 3
pd.set_option('display.max_rows', 7) # pd.options.display.max_rows = 3
pd.set_option('display.float_format', '{:.2f}'.format) # = pd.options.display.float_format = '{:.0f}'.format


#### Common Parameters

In [2]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Referer": "http://main.tsetmc.com/StaticContent/WebServiceHelp",
}
cookie_jar = {"ASP.NET_SessionId": "wa40en1alwxzjnqehjntrv5j"}
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']
# currentDateTime = datetime.now().strftime("%Y-%m-%d %H-%M-%S %p")

last_date = fetch.api_last_possible_deven()
instruments = fetch.instruments()
identities = fetch.identities()

insCodes = list(instruments.index)
insCodes_indices_excluded = list(instruments[instruments['cComVal'] != '6'].index)

print(last_date)

20230910


In [3]:
## Getting Prices
daily_prices = fetch.get_daily_prices(insCodes)
print(f'{len(daily_prices.keys())} Tickers have history.')

1934 Tickers have history.


## Categorization

In [4]:
def filter_tickers_by_industry(cSecVal: int) -> list:
    return list(identities[identities['sector_cSecVal'] == cSecVal].index)

In [5]:
khodroi = filter_tickers_by_industry(34)

## Volume Filter

In [10]:

def volume_filter(insCodes: list) -> list:
    
    daily_prices = fetch.get_daily_prices(insCodes)
    
    list_of_dfs = []
    ticker_names= []
    # TODO: درست کردن فیلتر «نمادهای معامله شده امروز/روز آخر کاری»ا
    for insCode in insCodes:
        last_row = daily_prices[insCode].iloc[-1]
        if (
            daily_prices[insCode].index[-1] == int(last_date) and
            last_row['<VOL>'] > last_row['<VOL> MA 3'] and 
            last_row['<VOL>'] > last_row['<VOL> MA 5'] and
            last_row['<VOL>'] > last_row['<VOL> MA 9'] and
            last_row['<VOL>'] > last_row['<VOL> MA 26'] and
            last_row['<VOL>'] > last_row['<VOL> MA 52'] and
            last_row['<VOL>'] > last_row['<VOL> MA 90']
        ):
            list_of_dfs.append(daily_prices[insCode])
            ticker_names.append(insCode)
            debug_log.info(f'{insCode} Checked.')
    print(len(ticker_names),'\n')
    return ticker_names


In [11]:
volume_filter(list(daily_prices.keys()))

77 



['490987973229371',
 '793710053482057',
 '1931037287459558',
 '2161110547458064',
 '3173544097113770',
 '3542690854557886',
 '4247709727327181',
 '4733285133017464',
 '5920901446678689',
 '6478064539164167',
 '7395271748414592',
 '9141577977527107',
 '9546211590069761',
 '9882348775152461',
 '10114441830266109',
 '10568944722570445',
 '11560002870991149',
 '12303918642491681',
 '13666407494621646',
 '15726796686853780',
 '16369313804633525',
 '17649149765849787',
 '18668624820225527',
 '18883380772506226',
 '20133434564923831',
 '20560887114747719',
 '22087269603540841',
 '22259718159702272',
 '23049019886587905',
 '23293437377896568',
 '24212636157410845',
 '27000326841257664',
 '30650426998863332',
 '30719054967088301',
 '30974710508383145',
 '31039212000825988',
 '33527290777160784',
 '33808206014018431',
 '34718633636164421',
 '35445515321658835',
 '42696242981550091',
 '43062880954780884',
 '43267179898797137',
 '43622578471330344',
 '46178280540110577',
 '49163985123560149',
 '50